# Ensemble with an MPI Application

This tutorial highlights the capability to run and monitor user
applications using a libEnsemble Executor.

Our calling script registers a compiled MPI program that simulates
electrostatic forces between a collection of particles. The simulator function
launches instances of this executable and reads the result from an output file.

libEnsemble's MPI Executor can automatically detect available MPI runners and 
resources, and by default, divides them equally among workers.

**Note that for notebooks** the multiprocessing start method should be set to
`fork` (default on Linux). To use with `spawn` (default on Windows and macOS),
use the `multiprocess` library.

Let's make sure libEnsemble is installed.

In [ ]:
!pip install libensemble

## Getting Started

**An MPI distribution is required to use this notebook**.

We recommend using `mpich`, but `Open MPI` can be used. If running in a hosted
notebook with Open MPI, the following my be required.

In [ ]:
import os

os.environ["OMPI_ALLOW_RUN_AS_ROOT"] = "1"
os.environ["OMPI_ALLOW_RUN_AS_ROOT_CONFIRM"] = "1"

The simulation source code ``forces.c`` can be obtained directly from the
libEnsemble repository.

Assuming MPI and its C compiler ``mpicc`` are available, obtain
``forces.c`` and compile it into an executable (``forces.x``) with:  

In [ ]:
import subprocess
import requests

# This line is not necessary if forces.c is present in the current directory.
url = "https://raw.githubusercontent.com/Libensemble/libensemble/main/libensemble/tests/scaling_tests/forces/forces_app/forces.c"
forces = requests.get(url)

open("./forces.c", "wb").write(forces.content)

# Compile the forces MPI executable.
subprocess.run("mpicc -O3 -o forces.x forces.c -lm".split())

### Clean Up

This function can be used to ensure any output from previous runs is cleaned up.

In [ ]:
# To rerun this notebook, we need to delete the ensemble directory.
import os
import shutil


def cleanup():
    output_files = ["ensemble.log", "libE_stats.txt"]
    for file_path in output_files:
        try:
            os.remove(file_path)
        except:
            pass
    try:
        shutil.rmtree("ensemble")
    except:
        pass

## Simulation Function

Our simulation function is where we'll use libEnsemble's executor to run our MPI application. We'll wait until this task's finished  and then send the results back to the manager.

Running the following cell will load the function `run_forces` into memory. In a standard Python environment (outside of a notebook), this function would be in a separate file (e.g., ``forces_simf.py``) and imported in your calling script.

In [ ]:
import numpy as np

# Optional status codes to display in libE_stats.txt for each gen or sim
from libensemble.message_numbers import WORKER_DONE, TASK_FAILED


def read_output(sim_specs):

    calc_status = 0

    statfile = "forces.stat"

    # Try loading final energy reading, set the sim's status
    try:
        data = np.loadtxt(statfile)
        final_energy = data[-1]
        calc_status = WORKER_DONE
    except Exception:
        final_energy = np.nan
        calc_status = TASK_FAILED

    # Define our output array,  populate with energy reading
    outspecs = sim_specs["out"]
    output = np.zeros(1, dtype=outspecs)
    output["energy"][0] = final_energy

    return output, calc_status


def run_forces(H, persis_info, sim_specs, libE_info):

    # Parse out num particles, from generator function
    particles = str(int(H["x"][0][0]))

    # num particles, timesteps, also using num particles as seed
    args = particles + " " + str(10) + " " + particles

    # Retrieve our MPI Executor
    exctr = libE_info["executor"]

    # Submit our forces app for execution.
    # Exclude num_procs to use all cores available to worker
    task = exctr.submit(app_name="forces", app_args=args, num_procs=1)

    # Block until the task finishes
    task.wait()

    output, calc_status = read_output(sim_specs)

    # Return final information to worker, for reporting to manager
    return output, persis_info, calc_status

The `run_forces` function retrieves the generated number of particles from ``H`` and
constructs an argument string for our launched application. The particle count doubles
up as a random number seed here.

After submitting the "forces" app for execution, a `Task` object is returned.
This object can be polled, killed, and evaluated in a variety of helpful ways.
For now, we're satisfied with waiting for the task to complete via `task.wait()`.

Our application produces a `forces.stat` file that contains energy outputs from the simulation.

To complete our simulation function, parse the last energy value from the output file into
a local output History array, and if successful, set the simulation function's exit
status ``calc_status`` to `WORKER_DONE`. Otherwise, send back `NAN` and a `TASK_FAILED`
status:

`calc_status` is an optional return value and will be displayed in the `libE_stats.txt` log file.

## Calling Script

Let's begin by writing our calling script to parameterize our simulation and
generation functions and run the ensemble.

In [ ]:
#!/usr/bin/env python
import os
import numpy as np
from pprint import pprint

# from forces_simf import run_forces  # Use is sim function is in a file

from libensemble import Ensemble
from libensemble.specs import ExitCriteria, GenSpecs, LibeSpecs, SimSpecs
from libensemble.gen_funcs.sampling import uniform_random_sample
from libensemble.executors import MPIExecutor

# Initialize MPI Executor
exctr = MPIExecutor()

# Register simulation executable with executor
sim_app = os.path.join(os.getcwd(), "forces.x")
exctr.register_app(full_path=sim_app, app_name="forces")

We instantiate our `MPIExecutor` in the calling script.

Registering an application is as easy as providing the full file-path and giving
it a memorable name. This Executor will later be retrieved within our simulation
function to launch the registered app.

Next define the `libE_specs`, `sim_specs` and `gen_specs` data structures:

In [ ]:
# Global settings - including creating directories for each simulation
libE_specs = LibeSpecs(
    nworkers=2,
    sim_dirs_make=True,
)

gen_specs = GenSpecs(
    gen_f=uniform_random_sample,
    outputs=[("x", float, (1,))],
    user={
        "lb": np.array([1000]),  # min particles
        "ub": np.array([3000]),  # max particles
        "gen_batch_size": 8,
    },
)

sim_specs = SimSpecs(
    sim_f=run_forces,
    inputs=["x"],  # Name of input for sim_f (defined in gen_specs.outputs)
    outputs=[("energy", float)],
)

We set ``exit_criteria`` initialize the ensemble and give each worker a random stream: 


In [ ]:
# Instruct libEnsemble to exit after this many simulations
exit_criteria = ExitCriteria(sim_max=8)

# Initialize ensemble object, passing executor.
ensemble = Ensemble(
    executor=exctr,
    libE_specs=libE_specs,
    gen_specs=gen_specs,
    sim_specs=sim_specs,
    exit_criteria=exit_criteria,
)

# Seed random streams for each worker, particularly for gen_f
ensemble.add_random_streams()

Now we are ready to run. The cleanup() function makes sure a re-run will work.

In [ ]:
# Clean up any previous outputs and launch libEnsemble
cleanup()
H, persis_info, flag = ensemble.run()

# See results
pprint(H[["sim_id", "x", "energy"]])

## Output files

That's it! As can be seen, with libEnsemble, it's relatively easy to get started with launching applications. Behind the scenes, libEnsemble evaluates default MPI runners and available resources and divides them among the workers.

This completes our calling script and simulation function.

Output files for each simulation will appear under the `ensemble` directory. Overall workflow information should appear in `libE_stats.txt` and `ensemble.log` as usual.

In [ ]:
# Print libE_stats.txt
with open("libE_stats.txt", "r") as file:
    print(file.read())

The file `ensemble.log` contains the MPI run lines for each simulation.

In [ ]:
# Show my run-lines
with open("ensemble.log", "r") as file:
    for line in file:
        if "Launching" in line:
            colon_index = line.index(":", line.index("Launching"))
            print(line[colon_index + 1 :].strip())

If you were running on a multi-node system, the nodes would be shown here also.

If you have multiple cores, and remove the `num_procs=1` from the `exctr.submit()` line,
then the cores on your system should be divided equally among the two workers. For this
you need at least one core per process.

**That concludes this tutorial.**

Each of these example files can be found in the repository in ``examples/tutorials/forces_with_executor``

For further experimentation, we recommend trying out this libEnsemble tutorial
workflow on a cluster or multi-node system, since libEnsemble can also manage
those resources and is developed to coordinate computations at huge scales.
Please feel free to contact us or open an issue on GitHub if this tutorial
workflow doesn't work properly on your cluster or other compute resource.

Exercise
--------
Using an input file for simulations.

Many applications read an input file instead of being given parameters directly on the run line.

We will use `jinja2` to parameterize a templated file for each simulation

You may need to install jinja2.

In [ ]:
!pip install jinja2

Now we can make a templated input file.

In [ ]:
# Define the content of the input file
input_content = """# Our input file - remember we are using particles as seed also for simplicity.
num_particles = {{particles}}
num_steps = 10
rand_seed = {{particles}}
"""

# Write the content to a file
with open("forces_input", "w") as file:
    file.write(input_content)

# Lets see our forces_input file
with open("forces_input", "r") as file:
    content = file.read()
    print(content)

During the run, libEnsemble will copy this templated input file to each simulation directory.

Now we will modify our simulation function that is run on each worker. We will read in the templated input file and then use `jinja2` to write it back to file with the parameters for the current simulation.

For `exctr.submit()` we remove `app_args` as **forces.x** will read from an input file called `forces_input` if it exists. If needed `app_args` could contain the name of the input file or any string to be placed after the application name on the run line.

In [ ]:
import numpy as np
import jinja2

# Optional status codes to display in libE_stats.txt for each gen or sim
from libensemble.message_numbers import WORKER_DONE, TASK_FAILED


def set_input_file_params(H, sim_specs, ints=False):
    """
    This is a general function to parameterize the input file with any inputs
    from sim_specs["in"]

    Often sim_specs_in["x"] may be multi-dimensional, where each dimension
    corresponds to a different input name in sim_specs["user"]["input_names"]).
    Effectively an unpacking of "x"
    """
    input_file = sim_specs["user"]["input_filename"]
    input_values = {}
    for i, name in enumerate(sim_specs["user"]["input_names"]):
        value = int(H["x"][0][i]) if ints else H["x"][0][i]
        input_values[name] = value
    with open(input_file, "r") as f:
        template = jinja2.Template(f.read())
    with open(input_file, "w") as f:
        f.write(template.render(input_values))


def run_forces_using_file(H, persis_info, sim_specs, libE_info):
    calc_status = 0

    # Construct input file for simulation
    input_file = sim_specs["user"]["input_filename"]
    set_input_file_params(H, sim_specs, ints=True)

    # Retrieve our MPI Executor
    exctr = libE_info["executor"]

    # Submit our forces app for execution.
    task = exctr.submit(app_name="forces", num_procs=1)  # app_args removed

    # Block until the task finishes
    task.wait(timeout=60)

    output, calc_status = read_output(sim_specs)

    # Return final information to worker, for reporting to manager
    return output, persis_info, calc_status

Time to modify our calling code, clean up, and rerun.

Assuming the original ensemble has been run above, we can just update `libE_specs` and `sim_specs` as below, and reset the random number seed for the generator (for reproducible output).

`sim_dir_copy_files` can be a full path or, as in our case, a relative path (it is in the current directory).

`sim_specs["user"]` fields are used in the simulation function to extract names of variables in the templated file.

In [ ]:
# Note the added option sim_dir_copy_files - a list of files to be copied to each simulation dir.
input_file = "forces_input"

# Add a field to libE_specs
ensemble.libE_specs.sim_dir_copy_files = [input_file]

ensemble.sim_specs = SimSpecs(
    sim_f=run_forces_using_file,
    inputs=["x"],  # Name of input for sim_f (defined in gen_specs.outputs)
    outputs=[("energy", float)],
    user={"input_filename": input_file, "input_names": ["particles"]},
)

# To reset random number seed in the generator
ensemble.add_random_streams()

# Clean up any previous outputs and launch libEnsemble
cleanup()
H, persis_info, flag = ensemble.run()

# See results
pprint(H[["sim_id", "x", "energy"]])

The templated `sim_dir_copy_files` option is a list of files to be copied to each simulation directory. 

Note: An alternative if there are multiple input files is to use the `sim_input_dir` option. The specified directory, and any files therein, is used as the basis for each simulation directory. 

Finally, lets have a look at the simulation directories that were created.

In [ ]:
! ls -l ensemble/sim*